# Task4

Index_X = FSR_for_force, FSR_for_coord

Index_y = force, x_coord, y_coord

Data = Splited by Subject

## Run result

https://wandb.ai/seokjin/FSR-prediction/groups/FSR_Trainable_2023-08-10_22-35-32/workspace?workspace=user-seokjin

## Experiment id

FSR_Trainable_2023-08-10_22-35-32

## Best metric (RMSE)

177.361

0.974

In [1]:
def define_searchspace(trial):
    model = trial.suggest_categorical('model', ['fsr_model.ANN'])
    if model == 'fsr_model.LSTM':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    elif model == 'fsr_model.CNN_LSTM':
        trial.suggest_categorical('model_args/cnn_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_categorical('model_args/lstm_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/cnn_num_layer', 1, 8)
        trial.suggest_int('model_args/lstm_num_layer', 1, 8)
    elif model == 'fsr_model.ANN':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    trial.suggest_categorical('criterion', ['torch.nn.MSELoss'])
    trial.suggest_categorical('optimizer', [
        'torch.optim.Adam',
        'torch.optim.NAdam',
        'torch.optim.Adagrad',
        'torch.optim.RAdam',
        'torch.optim.SGD',
    ])
    trial.suggest_float('optimizer_args/lr', 1e-5, 1e-1, log=True)
    imputer = trial.suggest_categorical('imputer', ['sklearn.impute.SimpleImputer'])
    if imputer == 'sklearn.impute.SimpleImputer':
        trial.suggest_categorical('imputer_args/strategy', [
            'mean',
            'median',
        ])
    trial.suggest_categorical('scaler', [ 
        'sklearn.preprocessing.StandardScaler',
        'sklearn.preprocessing.MinMaxScaler',
        'sklearn.preprocessing.RobustScaler',
    ])
    return {
        'index_X': ['FSR_for_force', 'FSR_for_coord'],
        'index_y': ['force', 'x_coord', 'y_coord'],
        'data_loader': 'fsr_data.get_index_splited_by_subject'
    }

In [2]:
import ray.tune
import ray.air
import ray.air.integrations.wandb
import ray.tune.schedulers
from fsr_trainable import FSR_Trainable
import ray.tune.search
import ray.tune.search.optuna

tuner = ray.tune.Tuner(
    trainable=ray.tune.with_resources(
        FSR_Trainable, {'cpu':2},
    ),
    tune_config=ray.tune.TuneConfig(
        num_samples=100,
        scheduler=ray.tune.schedulers.ASHAScheduler(
            max_t=100,
            grace_period=1,
            reduction_factor=2,
            brackets=1,
            metric='metric',
            mode='min',
        ),
        search_alg=ray.tune.search.optuna.OptunaSearch(
            space=define_searchspace,
            metric='metric',
            mode='min',
        ),
    ), 
    run_config=ray.air.RunConfig(
        callbacks=[
            ray.air.integrations.wandb.WandbLoggerCallback(project='FSR-prediction'),
        ],
        checkpoint_config=ray.air.CheckpointConfig(
            num_to_keep=3,
            checkpoint_score_attribute='metric',
            checkpoint_score_order='min',
            checkpoint_frequency=5,
            checkpoint_at_end=True,
        ),
    ), 
)

[I 2023-08-10 22:35:32,849] A new study created in memory with name: optuna


In [3]:
results = tuner.fit()

2023-08-10 22:35:35,008	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-08-10 22:35:37,100	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


2023-08-10 22:35:37,153	INFO wandb.py:320 -- Already logged into W&B.


Trial name,date,done,hostname,iterations_since_restore,mae_coord,mae_force,mape_coord,mape_force,metric,node_ip,pid,rmse_coord,rmse_force,time_since_restore,time_this_iter_s,time_total_s,timestamp,tmae_coord,tmae_force,tmape_coord,tmape_force,training_iteration,trial_id,trmse_coord,trmse_force
FSR_Trainable_00cc9211,2023-08-10_22-48-51,True,DESKTOP-0P789CI,64,4.46879,632.662,1.11428,8.32014e+17,0.65263,172.26.215.93,149034,2.43054,438.157,24.4484,0.631907,24.4484,1691675331,0.95916,0.252381,8.01392e+13,3.71883e+14,64,00cc9211,0.486823,0.165807
FSR_Trainable_0350f91f,2023-08-10_22-40-53,True,DESKTOP-0P789CI,4,5.50522,766.778,1.36742,1.24086e+18,0.742581,172.26.215.93,140923,2.71,526.43,2.64916,0.563669,2.64916,1691674853,1.13541,0.349622,8.29604e+13,6.92473e+14,4,0350f91f,0.523601,0.21898
FSR_Trainable_0af22b82,2023-08-10_22-37-29,True,DESKTOP-0P789CI,1,7.16495,1381.82,1.54647,1.34095e+17,56.2364,172.26.215.93,135944,3.73784,1192.11,0.684115,0.684115,0.684115,1691674649,48.2198,8.14161,1.36862e+16,5.23546e+15,1,0af22b82,48.2253,8.01112
FSR_Trainable_0be5760d,2023-08-10_22-41-36,True,DESKTOP-0P789CI,2,19.1085,1335.66,3.30393,1.86761e+18,1.66398,172.26.215.93,141931,6.75541,937.552,0.907253,0.307523,0.907253,1691674896,4.06787,0.53555,5.60158e+13,1.00784e+15,2,0be5760d,1.36343,0.300552
FSR_Trainable_0d1de52d,2023-08-10_22-39-40,True,DESKTOP-0P789CI,1,23.8971,1894.99,3.68499,2.24457e+18,2.22064,172.26.215.93,139298,7.90143,1361.63,0.736976,0.736976,0.736976,1691674780,5.54185,0.657647,3.75797e+13,9.59173e+14,1,0d1de52d,1.8092,0.411442
FSR_Trainable_116c5420,2023-08-10_22-49-25,True,DESKTOP-0P789CI,100,4.42188,627.988,1.10052,8.18704e+17,0.649558,172.26.215.93,149398,2.41949,436.472,41.3902,0.463696,41.3902,1691675365,0.950661,0.247729,7.90436e+13,3.5731e+14,100,116c5420,0.485795,0.163763
FSR_Trainable_13115552,2023-08-10_22-41-32,True,DESKTOP-0P789CI,64,5.42585,739.719,1.17501,1.23462e+18,0.696955,172.26.215.93,141231,2.60987,475.263,23.9775,0.265702,23.9775,1691674892,1.13926,0.315835,7.36685e+13,6.14835e+14,64,13115552,0.512455,0.1845
FSR_Trainable_14fdb345,2023-08-10_22-38-00,True,DESKTOP-0P789CI,8,5.89484,749.007,1.37012,1.38367e+18,0.721609,172.26.215.93,136666,2.69268,451.087,3.77932,0.337322,3.77932,1691674680,1.2227,0.349352,6.42736e+13,7.93542e+14,8,14fdb345,0.525736,0.195873
FSR_Trainable_17ad2c46,2023-08-10_22-53-38,True,DESKTOP-0P789CI,1,22.9792,2687.17,3.99761,3.50714e+18,2.4335,172.26.215.93,154948,8.11512,1751.45,0.453102,0.453102,0.453102,1691675618,5.25306,0.973435,4.69889e+13,1.54423e+15,1,17ad2c46,1.86678,0.566717
FSR_Trainable_1a2e7ffb,2023-08-10_22-41-13,True,DESKTOP-0P789CI,1,5.50955,844.833,1.25407,7.98399e+08,7.33338,172.26.215.93,141461,3.02712,597.814,0.724885,0.724885,0.724885,1691674873,9.81804,2.61905,4.30316e+15,139.198,1,1a2e7ffb,5.29239,2.04099


(_WandbLoggingActor pid=133856) wandb: Currently logged in as: seokjin. Use `wandb login --relogin` to force relogin
(_WandbLoggingActor pid=133856) wandb: wandb version 0.15.8 is available!  To upgrade, please run:
(_WandbLoggingActor pid=133856) wandb:  $ pip install wandb --upgrade
(_WandbLoggingActor pid=133856) wandb: Tracking run with wandb version 0.15.4
(_WandbLoggingActor pid=133856) wandb: Run data is saved locally in /home/seokj/ray_results/FSR_Trainable_2023-08-10_22-35-32/FSR_Trainable_e8eecf5d_1_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_subject,imputer=sklearn_impute_Simpl_2023-08-10_22-35-37/wandb/run-20230810_223548-e8eecf5d
(_WandbLoggingActor pid=133856) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=133856) wandb: Syncing run FSR_Trainable_e8eecf5d
(_WandbLoggingActor pid=133856) wandb: ⭐️ View project at https://wandb.ai/seokjin/FSR-prediction
(_WandbLoggingActor pid=133856) wandb: 🚀 View run at https://wandb.ai/se